# Example: N-Ary Recombining Lattice Models
In this example, we demonstrate how to implement an N-ary recombining lattice model. An N-ary lattice allows each node to branch into N possible states at each time step, and the lattice recombines at certain points to reduce complexity.

> __Learning Objectives__
>
> In this example, students will learn to: 
> * **N-ary Lattice Construction**: Build multi-branch lattice models from historical stock data using more than two possible price movements per time step. We estimate growth factors and probabilities for each branch to create realistic price trees.
> * **Recombining Tree Implementation**: Construct lattice models that recombine at specific levels to manage computational complexity while maintaining accuracy. We populate the tree with prices and track connectivity between nodes across different time periods.
> * **Price Distribution Analysis**: Analyze possible future price outcomes at specific time horizons using the N-ary lattice structure. We compute probabilities for each possible price level and visualize the distribution of potential returns.

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

>__Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [3]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

# hack:
using UnicodePlots

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

### Data
We gathered daily open-high-low-close (OHLC) data for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2014` until `12-31-2024`, along with data for a few exchange-traded funds and volatility products during that time period. 

Let's load the `original_dataset::DataFrame` by calling [the `MyTrainingMarketDataSet()` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/data/#VLQuantitativeFinancePackage.MyTrainingMarketDataSet) and remove firms that do not have the maximum number of trading days. The cleaned dataset $\mathcal{D}$ will be stored in the `dataset` variable.

In [6]:
original_dataset = MyTrainingMarketDataSet() |> x-> x["dataset"];

Not all tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquisition or delisting events. Let's collect only those tickers with the maximum number of trading days.

First, let's compute the number of records for a firm that we know has the maximum value, e.g., `AAPL`, and save that value in the `maximum_number_trading_days::Int64` variable:

In [8]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow # nrow? (check out: DataFrames.jl)

2767

Now, let's iterate through our data and collect only tickers with `maximum_number_trading_days` records. We'll save that data in the `dataset::Dict{String,DataFrame}` variable:

In [10]:
dataset = let

    # initialize -
    dataset = Dict{String, DataFrame}();

    # iterate through the dictionary; we can't guarantee a particular order
    for (ticker, data) ∈ original_dataset  # we get each (K, V) pair!
        if (nrow(data) == maximum_number_trading_days) # what is this doing?
            dataset[ticker] = data;
        end
    end
    dataset; # return
end;

Finally, let's get a list of the firms in our cleaned dataset and sort them alphabetically. We store the sorted firm ticker symbols in the `list_of_tickers::Array{String,1}` variable.

In [12]:
list_of_tickers = keys(dataset) |> collect |> sort # list of firm "ticker" symbols in alphabetical order

424-element Vector{String}:
 "A"
 "AAL"
 "AAP"
 "AAPL"
 "ABBV"
 "ABT"
 "ACN"
 "ADBE"
 "ADI"
 "ADM"
 "ADP"
 "ADSK"
 "AEE"
 ⋮
 "WST"
 "WU"
 "WY"
 "WYNN"
 "XEL"
 "XOM"
 "XRAY"
 "XYL"
 "YUM"
 "ZBRA"
 "ZION"
 "ZTS"

### Constants
Finally, let's set some constants we'll use later in this notebook. The comments describe the constants, their units, and permissible values.

In [161]:
TSIM = 8; # number of trading days to simulate
Δt = (1.0/252); # step size: 1 trading day in units of years
r̄ = 0.05; # risk-free rate (annualized)
n = 9; # number of branches in the N-ary lattice

#### Helper Functions
We define a few helper functions that we'll use later in this notebook.
* The `nodes_at_level(...)` function computes the number of nodes at a given level in an N-ary recombining lattice.
* The `level_offset(...)` function computes the offset to the first node at a given level in an N-ary recombining lattice.

In [163]:
nodes_at_level(i::Integer, n::Integer) = binomial(i + n - 1, i);
level_offset(i::Integer, n::Integer) = i == 0 ? 0 : binomial(i + n - 1, i - 1); # start of level i

___

## Task 1: Compute lattice parameters and future prices from historical data
In this task, we estimate the parameters needed to construct an N-ary recombining lattice model from historical data.

First, we specify the firm we want to analyze. Let's store this in the `selected_firm_ticker::String` variable, get the index of that firm in our `list_of_tickers::Array{String,1}` variable, and extract the firm's historical data from our `dataset::Dict{String,DataFrame}` variable. We'll store the firm's historical data in the `selected_firm_data::DataFrame` variable.

In [166]:
# selected_firm_ticker = rand(list_of_tickers);
selected_firm_ticker = "AAPL"
selected_firm_index = findfirst(x-> x == selected_firm_ticker, list_of_tickers);
selected_firm_data = dataset[selected_firm_ticker];

Next, we specify the `start_index` as the trading day index in the dataset, which will serve as the tree's starting point or `L = 0`. Finally, we set the variable `Sₒ`, which corresponds to the initial price per share at the root of the tree; we use the [volume-weighted average price (VWAP)](https://en.wikipedia.org/wiki/Volume-weighted_average_price) as the initial condition:

In [168]:
#start_index = rand(1:(maximum_number_trading_days - TSIM - 1))
start_index = 1465; # start index for the trading days
stop_index = start_index + TSIM
println("Visualize Firm-$(selected_firm_index) between trading days ($(start_index) -> $(stop_index))")

Visualize Firm-4 between trading days (1465 -> 1473)


We use daily data; thus, the natural time frame between $S_{j-1}$ and $S_{j}$ is a single day. However, it is more convenient to use an annualized value for the $\mu$ parameter; thus, we let $\Delta{t} = 1/252$, i.e., the fraction of a year that corresponds to a single trading day.

The [`log_growth_matrix(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.log_growth_matrix) takes the cleaned dataset $\mathcal{D}$, which contains $T$ days of data for each firm, a list of firms $\mathcal{L}$, and computes the growth rate values for each firm as a function of time. 

The data is returned as a $(T - 1)\times\dim\mathcal{L}$ array (time on the rows, firm $i$ on the columns). We store the data in the `log_growth_array::Array{Float64,1}` variable:

In [170]:
log_growth_array = log_growth_matrix(dataset, selected_firm_ticker); # array holding growth rate time series

In [171]:
UnicodePlots.histogram(log_growth_array, nbins=21, closed=:left)

                  ┌                                        ┐ 
   [-22.0, -20.0) ┤▎ 3                                       
   [-20.0, -18.0) ┤  0                                       
   [-18.0, -16.0) ┤▏ 1                                       
   [-16.0, -14.0) ┤▎ 3                                       
   [-14.0, -12.0) ┤▌ 11                                      
   [-12.0, -10.0) ┤▋ 13                                      
   [-10.0,  -8.0) ┤█▍ 30                                     
   [ -8.0,  -6.0) ┤███▍ 73                                   
   [ -6.0,  -4.0) ┤███████▎ 159                              
   [ -4.0,  -2.0) ┤█████████████▋ 300                        
   [ -2.0,   0.0) ┤█████████████████████████████▍ 649        
   [  0.0,   2.0) ┤███████████████████████████████████  773  
   [  2.0,   4.0) ┤███████████████████▌ 431                  
   [  4.0,   6.0) ┤████████▊ 194                             
   [  6.0,   8.0) ┤███▏ 67                                   
   [  8.

We've developed the [`build_nary_lattice_from_growth_rate(...)` function](src/Split.jl) to estimate the parameters needed to construct an N-ary recombining lattice model from historical data. The function takes the following arguments:
* `log_growth_array::Array{Float64,2}`: The array of log growth rates for the firms in the dataset.
* `n::Int64`: The number of branches at each node in the N-ary lattice, e.g., `n=2` for a binary lattice, `n=3` for a ternary lattice, etc.
* `dt::Float64`: The time step between levels in the lattice, expressed in years.
* `method::Symbol`: The method used to estimate the lattice parameters; `:quantile` (equal-mass bins) or `:equalwidth` (uniform in growth rate space).

We'll save the output in the `result::NamedTuple` variable.

In [173]:
result = build_nary_lattice_from_growth_rate(log_growth_array; n = n, dt = Δt, method = :equalwidth)

(edges = [-21.211458354728737, -16.44284317108319, -11.674227987437643, -6.9056128037920965, -2.13699762014655, 2.631617563498997, 7.4002327471445435, 12.16884793079009, 16.937463114435637, 21.706078298081184], avg_factor = [0.9255544819471232, 0.9506296008770745, 0.9669074042634733, 0.9844771088544999, 1.001280551609203, 1.0167980876086244, 1.0356614371941943, 1.0572440704710517, 1.0811286006001757], freq = [0.0014461315979754157, 0.006869125090383225, 0.0227765726681128, 0.175704989154013, 0.5824295010845987, 0.18257411424439624, 0.0227765726681128, 0.004338394793926247, 0.0010845986984815619], counts = [4, 19, 63, 486, 1611, 505, 63, 12, 3], labels = ["S1", "S2", "S3", "S4", "S5", "S6", "S7", "S8", "S9"], method = :equalwidth, dt = 0.003968253968253968, N = 2766)

Let's check out the fields in the `result::NamedTuple` variable.

In [175]:
typeof(result) |> T-> fieldnames(T) # check out the fields of the returned struct

(:edges, :avg_factor, :freq, :counts, :labels, :method, :dt, :N)

The `avg_factor::Array{Float64,1}` field contains the average growth factors associated with each branch at a node in the lattice (arranged in ascending order).

In [177]:
result.avg_factor

9-element Vector{Float64}:
 0.9255544819471232
 0.9506296008770745
 0.9669074042634733
 0.9844771088544999
 1.001280551609203
 1.0167980876086244
 1.0356614371941943
 1.0572440704710517
 1.0811286006001757

The `freq::Array{Float64,1}` field contains the frequencies associated with each branch at a node in the lattice. 

In [179]:
result.freq

9-element Vector{Float64}:
 0.0014461315979754157
 0.006869125090383225
 0.0227765726681128
 0.175704989154013
 0.5824295010845987
 0.18257411424439624
 0.0227765726681128
 0.004338394793926247
 0.0010845986984815619

In [180]:
print_lattice(result)

n-ary lattice (method=equalwidth, Δt=0.003968253968253968, N=2766)
State μ-bin [low, high)               avg factor    freq      count
S1    [-21.211458 , -16.442843)       0.925554      0.001446  4
S2    [-16.442843 , -11.674228)       0.95063       0.006869  19
S3    [-11.674228 , -6.905613)        0.966907      0.022777  63
S4    [-6.905613 , -2.136998)         0.984477      0.175705  486
S5    [-2.136998 , 2.631618)          1.001281      0.58243   1611
S6    [2.631618 , 7.400233)           1.016798      0.182574  505
S7    [7.400233 , 12.168848)          1.035661      0.022777  63
S8    [12.168848 , 16.937463)         1.057244      0.004338  12
S9    [16.937463 , 21.706078]         1.081129      0.001085  3


Now, let's build and populate our N-ary recombining lattice model. We'll store the model in the `my_nary_lattice_model::MyGeneralAdjacencyRecombiningCommodityPriceTree` variable.

In [182]:
my_nary_lattice_model = let

    # initialize -
    model = nothing;
    Sₒ = selected_firm_data[start_index,:volume_weighted_average_price];
    Δ = result.avg_factor |> reverse; # average growth factors (up to down)

    # build an empty model -
    model = build(MyGeneralAdjacencyRecombiningCommodityPriceTree, (
        n = n,
        h = TSIM, # how many days to simulate 
    ));

    # populate the data in the model -
    model = populate(model, Sₒ, Δ);

    # print -
    println("Starting price: $(Sₒ) USD for firm $(selected_firm_ticker)");

    model; # return
end;

Starting price: 62.0886 USD for firm AAPL


What's in the `my_nary_lattice_model::MyGeneralAdjacencyRecombiningCommodityPriceTree` instance?

In [184]:
typeof(my_nary_lattice_model) |> T-> fieldnames(T) # check out the fields of the returned struct

(:data, :connectivity, :h, :n)

Let's look at the `connectivity::Dict{Int64,Array{Int64,1}}` field, which contains the adjacency list for the lattice.

In [186]:
my_nary_lattice_model.data

Dict{Int64, NamedTuple} with 24310 entries:
  11950 => (price = 96.8367, path = [5, 0, 2, 0, 0, 1, 0, 0, 0])
  1703  => (price = 53.8636, path = [0, 0, 0, 0, 2, 0, 2, 0, 1])
  12427 => (price = 71.1348, path = [1, 0, 2, 1, 2, 2, 0, 0, 0])
  7685  => (price = 70.2225, path = [2, 0, 1, 2, 0, 0, 0, 2, 0])
  18374 => (price = 64.2087, path = [0, 1, 1, 2, 2, 1, 0, 0, 1])
  3406  => (price = 67.421, path = [0, 3, 0, 1, 0, 0, 0, 2, 0])
  23970 => (price = 45.6068, path = [0, 0, 0, 2, 1, 0, 1, 0, 4])
  1090  => (price = 58.2436, path = [0, 0, 0, 1, 0, 3, 1, 0, 0])
  2015  => (price = 90.9823, path = [4, 0, 2, 0, 0, 0, 0, 0, 0])
  18139 => (price = 72.0746, path = [0, 4, 0, 0, 3, 0, 0, 0, 1])
  17088 => (price = 70.4567, path = [0, 5, 0, 0, 0, 0, 0, 3, 0])
  11280 => (price = 53.9374, path = [1, 1, 1, 0, 0, 0, 0, 0, 4])
  16805 => (price = 59.9559, path = [1, 0, 1, 0, 2, 1, 1, 2, 0])
  3220  => (price = 66.2639, path = [1, 0, 2, 0, 1, 0, 1, 1, 0])
  11251 => (price = 46.1333, path = [0, 0, 1, 0

The `data::Dict{Int64, NamedTuple}` field contains a NamedTuple for each node in the lattice, storing relevant information such as the node's price, time, and other attributes.
* The `price::Float64` field contains the price at each node in the lattice.
* The `path::Array{Int64,1}` field contains the path taken to reach each node in the lattice, represented as an array of integers.

Let's take a look at the data.

In [188]:
my_nary_lattice_model.data

Dict{Int64, NamedTuple} with 24310 entries:
  11950 => (price = 96.8367, path = [5, 0, 2, 0, 0, 1, 0, 0, 0])
  1703  => (price = 53.8636, path = [0, 0, 0, 0, 2, 0, 2, 0, 1])
  12427 => (price = 71.1348, path = [1, 0, 2, 1, 2, 2, 0, 0, 0])
  7685  => (price = 70.2225, path = [2, 0, 1, 2, 0, 0, 0, 2, 0])
  18374 => (price = 64.2087, path = [0, 1, 1, 2, 2, 1, 0, 0, 1])
  3406  => (price = 67.421, path = [0, 3, 0, 1, 0, 0, 0, 2, 0])
  23970 => (price = 45.6068, path = [0, 0, 0, 2, 1, 0, 1, 0, 4])
  1090  => (price = 58.2436, path = [0, 0, 0, 1, 0, 3, 1, 0, 0])
  2015  => (price = 90.9823, path = [4, 0, 2, 0, 0, 0, 0, 0, 0])
  18139 => (price = 72.0746, path = [0, 4, 0, 0, 3, 0, 0, 0, 1])
  17088 => (price = 70.4567, path = [0, 5, 0, 0, 0, 0, 0, 3, 0])
  11280 => (price = 53.9374, path = [1, 1, 1, 0, 0, 0, 0, 0, 4])
  16805 => (price = 59.9559, path = [1, 0, 1, 0, 2, 1, 1, 2, 0])
  3220  => (price = 66.2639, path = [1, 0, 2, 0, 1, 0, 1, 1, 0])
  11251 => (price = 46.1333, path = [0, 0, 1, 0

## Task 2: Visualize the price distribution at a given level in the lattice
In this task, we visualize the price distribution at a specified level in the N-ary recombining lattice. We'll also compute the probability of the price at each node in the final level of the lattice.

Let's start by specifying the tree level we want to analyze and identifying the nodes at that level. We'll store the nodes at the specified level in the `nodes_at_tree_level::Array{Int64,1}` variable.

In [190]:
nodes_at_tree_level = let

    # initialize -
    l = TSIM; # level
    start = level_offset(l, n);
    stop = start + nodes_at_level(l, n) - 1;

    # compute the nodes indices at level l -
    nodes = range(start, stop=stop, step=1) |> collect;
end

12870-element Vector{Int64}:
 11440
 11441
 11442
 11443
 11444
 11445
 11446
 11447
 11448
 11449
 11450
 11451
 11452
     ⋮
 24298
 24299
 24300
 24301
 24302
 24303
 24304
 24305
 24306
 24307
 24308
 24309

Next, we compute the price and probability data for each node at the specified level. We extract the price from each node and calculate the probability of reaching that node using the multinomial distribution.

In [192]:
data_array_at_level = let
   
    # initialize -
    number_of_nodes = length(nodes_at_tree_level);
    results_array = Array{Float64, 2}(undef, number_of_nodes, 2);
    Δ = result.avg_factor |> reverse; # average growth factors (up to down)
    p = result.freq |> reverse; # real-world probabilities (up to down)
    d = Multinomial(TSIM, p)

    for i ∈ 1:number_of_nodes
        
        # get node index -
        j = nodes_at_tree_level[i];
        nodemodel = my_nary_lattice_model.data[j];

        # get the price from the node model -
        price = nodemodel.price;

        # ok: let's compute the probability of reaching this node -
        path = nodemodel.path; # path to reach this node
       
        # capture the results -
        results_array[i, 1] = price;
        results_array[i, 2] = pdf(d, path);
    end

    tmp = results_array[:,1];
    sorted_results_array = sortperm(tmp) |> I-> results_array[I, :]; # sort by price
end;

__Check:__ The sum of the probabilities at each level should equal `1.0`. Let's verify this for our specified level:

In [194]:
@assert data_array_at_level[:,2] |> sum ≈ 1.0 # should be 1.0

Let's visualize the possible price distribution at the specified level:

In [196]:
UnicodePlots.histogram(data_array_at_level[:,1], nbins=9, closed=:left)

                  ┌                                        ┐ 
   [ 30.0,  40.0) ┤▌ 62                                      
   [ 40.0,  50.0) ┤█████████▊ 1 264                          
   [ 50.0,  60.0) ┤██████████████████████████████▋ 3 907     
   [ 60.0,  70.0) ┤█████████████████████████████████  4 214  
   [ 70.0,  80.0) ┤██████████████████▍ 2 343                 
   [ 80.0,  90.0) ┤██████▌ 832                               
   [ 90.0, 100.0) ┤█▋ 209                                    
   [100.0, 110.0) ┤▍ 35                                      
   [110.0, 120.0) ┤▏ 4                                       
                  └                                        ┘ 
                                   Frequency                 

Now, let's compute the (multistep) growth rate distributions at the specified level in the lattice. We'll store the data in the `growth_rate_array_at_level::Array{Float64,1}` variable.

In [198]:
growth_rate_array_at_level = let

    # initialize -
    Sₒ = selected_firm_data[start_index,:volume_weighted_average_price];
    number_of_nodes = length(nodes_at_tree_level);
    results_array = Array{Float64, 1}(undef, number_of_nodes);

    for i ∈ 1:number_of_nodes
        
        # get node index -
        j = nodes_at_tree_level[i];
        nodemodel = my_nary_lattice_model.data[j];

        # get the price from the node model -
        price = nodemodel.price;

        # compute the growth rate from the starting price -
        growth_rate = (1/Δt)*log(price/Sₒ);
       
        # capture the results -
        results_array[i] = growth_rate;
    end

    results_array; # return
end;

Let's visualize the growth rate distribution at the specified level:

In [200]:
UnicodePlots.histogram(growth_rate_array_at_level, nbins=21, closed=:left)

                    ┌                                        ┐ 
   [-160.0, -140.0) ┤▏ 5                                       
   [-140.0, -120.0) ┤▍ 27                                      
   [-120.0, -100.0) ┤█▌ 100                                    
   [-100.0,  -80.0) ┤████▍ 286                                 
   [ -80.0,  -60.0) ┤█████████▌ 640                            
   [ -60.0,  -40.0) ┤█████████████████▋ 1 187                  
   [ -40.0,  -20.0) ┤██████████████████████████▍ 1 767         
   [ -20.0,    0.0) ┤████████████████████████████████▌ 2 189   
   [   0.0,   20.0) ┤█████████████████████████████████  2 219  
   [  20.0,   40.0) ┤███████████████████████████▊ 1 872        
   [  40.0,   60.0) ┤███████████████████▌ 1 314                
   [  60.0,   80.0) ┤███████████▎ 750                          
   [  80.0,  100.0) ┤█████▎ 348                                
   [ 100.0,  120.0) ┤█▊ 127                                    
   [ 120.0,  140.0) ┤▌ 32               

___

## Summary
In this notebook, we demonstrated how to construct and analyze N-ary recombining lattice models for stock price evolution using historical data.

> __Key takeaways__
>
> * **N-ary Lattice Construction**: We built multi-branch lattice models that capture more nuanced price movements than traditional binary trees, using three possible states (up, neutral, down) at each node.
> * **Historical Parameter Estimation**: We extracted growth factors and probabilities directly from S&P 500 log returns using equal-width binning to create realistic transition probabilities that reflect actual market behavior.
> * **Recombining Tree Efficiency**: We implemented lattice models that recombine at each level to maintain computational efficiency while preserving essential price dynamics and probability distributions.

Understanding how to build N-ary lattice models provides a more flexible framework for modeling complex financial market movements and price distributions.

___

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy, is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.